In [1]:
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
import pickle
f = open("../data/ava/signif_genomic_feat_cond_json.pkl", 'rb')
signif_feat_cond_json = pickle.load(f)
signif_feat_cond_json[0:5]

[{'mutated features': 'C0343',
  'condition': 'LB',
  'odd ratio': 6.6221624248746425,
  'p value': 1.3372836639814812e-46},
 {'mutated features': 'C0343',
  'condition': 'bla-TEM-1',
  'odd ratio': 6.137157987643424,
  'p value': 0.0016785684834465873},
 {'mutated features': 'C0343',
  'condition': 'ceftazidime',
  'odd ratio': 5.68303165550872,
  'p value': 1.034583052014539e-09},
 {'mutated features': 'C0343',
  'condition': 'pBGT',
  'odd ratio': 4.234702351426518,
  'p value': 0.0001059567713938109},
 {'mutated features': 'C0343',
  'condition': 'ΔtpiA',
  'odd ratio': 6.551766368346679,
  'p value': 0.001269592344262909}]

In [3]:
all_muts_df = pd.read_pickle("../data/ava/4_10_with_uniq_midpts.pkl")
all_muts_df.shape

(3921, 54)

In [4]:
def get_signif_mut_conds(feat_name):
    signif_mut_conds = []
    for signif_mut_cond in signif_feat_cond_json:
        if signif_mut_cond['mutated features'] == feat_name:
            signif_mut_conds.append(signif_mut_cond)
    return signif_mut_conds


temp_df = all_muts_df.copy()
temp_df["genomic features"] = temp_df.apply(lambda r: str(r["genomic features"]), axis=1)

cond_mut_feat_assoc_df = pd.DataFrame()
for _, mut in all_muts_df.iterrows():
    for feat in mut["genomic features"]:
        signif_mut_conds = get_signif_mut_conds(feat["name"])
        feat_name = str(feat["name"])
        df = temp_df[temp_df["genomic features"].str.contains(feat_name)]
        mut_cnt = len(df)
        exp_cnt = len(df["exp"].unique())
        for d in signif_mut_conds:
            d["mutation count"] = mut_cnt
            d["experiment count"] = exp_cnt
            cond_mut_feat_assoc_df = cond_mut_feat_assoc_df.append(d, ignore_index=True)

cond_mut_feat_assoc_df = cond_mut_feat_assoc_df.rename(columns={"mutated features": "mutated features"})
cond_mut_feat_assoc_df = cond_mut_feat_assoc_df.drop_duplicates()
display(len(cond_mut_feat_assoc_df), cond_mut_feat_assoc_df.head())

3320

,condition,experiment count,mutated features,mutation count,odd ratio,p value
0,glutaric acid,6.0,nagA,9.0,31.694192,1.512914e-04
1,ΔtpiA glucose M9 pre-evolved,6.0,nagA,9.0,29.964408,2.320996e-03
2,42 celsius,2.0,clsA,3.0,48.234483,2.162612e-02
3,42 celsius,37.0,rph-pyrE attenuator terminator,124.0,7.426796,2.780666e-04
4,CaCl2(0.1),37.0,rph-pyrE attenuator terminator,124.0,1.730867,6.520830e-08


In [5]:
cond_cnt_d = {"none": 0}
for _, mut in all_muts_df.iterrows():
    for feat in mut["genomic features"]:
        if len(feat["significantly associated conditions"]) > 0:
            for cond in feat["significantly associated conditions"]:
                if cond in cond_cnt_d.keys():                   
                    cond_cnt_d[cond] += 1
                else:
                    cond_cnt_d[cond] = 1
        else:
            cond_cnt_d["none"] += 1

cond_cnt_d

{'none': 1352,
 '42 celsius': 47,
 'NH4Cl(1)': 446,
 'M9': 923,
 'WT': 680,
 'glucose(4)': 592,
 'KH2PO4(3) Na2HPO4(6.8)': 933,
 'CaCl2(0.1)': 923,
 'MgSO4(0.24)': 933,
 'U-13C glucose(2)': 45,
 'NaCl(0.5g/L) trace elements': 1380,
 'ethylene glycol(30mM)': 60,
 'E. coli K-12 MG1655 pCA24N-fucO': 60,
 'chloramphenicol(25)': 60,
 '30 celsius': 215,
 'glycerol(2.5)': 150,
 'glucose(2)': 1500,
 'FeSO4(20uM) sodium citrate(10mM)': 18,
 'LB': 1280,
 'isobutanol(8g/L)': 205,
 'JCL260': 205,
 'glucose(3)': 211,
 'D-2-deoxyribose': 59,
 'D-arabinose': 13,
 'D-lyxose': 22,
 'm-tartrate': 131,
 'monomethyl succinate': 13,
 'bla-TEM-1': 196,
 'ceftazidime': 970,
 'pBGT': 648,
 'chloramphenicol': 51,
 'doxycycline': 44,
 'trimethoprim': 23,
 'glucose-evolved BOP1000': 8,
 'PQ(0.8mM)': 8,
 'Δpgi': 86,
 "glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1)": 25,
 'ΔsdaA ΔsdaB ΔtdcG ΔglyA': 25,
 'acetate(4)': 24,
 'glucose(4) or acetate(4)': 76,
 'glucose(4) or glycerol(2.5)': 2

In [6]:
signif_odds_assoc_mat = pd.DataFrame(0.0,
    index=cond_mut_feat_assoc_df["mutated features"].unique(),
    columns=cond_mut_feat_assoc_df["condition"].unique()
).astype(float)

for _, r in cond_mut_feat_assoc_df.iterrows():
    if r["p value"] <= 0.05:
        signif_odds_assoc_mat.at[r["mutated features"], r["condition"]] = r["odd ratio"]
signif_odds_assoc_mat

,glutaric acid,ΔtpiA glucose M9 pre-evolved,42 celsius,CaCl2(0.1),FeSO4(20uM) sodium citrate(10mM),GMOS,H. sapien pgi,H. sapien tpiA,KH2PO4(3) Na2HPO4(6.8),M9,MgSO4(0.24),NH4Cl(1),U-13C glucose(2),V. cholerae tpiA,WT,adipic acid,chloramphenicol,glucose(4),glucose(4) or acetate(4),glucose(4) or glycerol(2.5),glucose(4) or xylose(4),hexamethylenediamine,n-butanol,xylose(4),(NH4)2SO4(1),(NH4)2SO4(1) MgSO4,30 celsius,K2HPO4(5.3) KH2PO4(2),glucose(1),thiaminne hydrochloride(0.3µM) Ga(NO3)3(100mg/L),2 3-butanediol,B. melitensis pgi,V. cholerae pgi,hexanoic acid,octanoic acid,C13H10N2O(0.168184),isobutyric acid,coumaric acid,D-arabinose,acetate(4),PQ(0.8mM),glucose-evolved BOP1000,D-lyxose,NaCl(0.5g/L) trace elements,P. aerophilum pgi,monomethyl succinate,glycerol(2.5),CaCl2(0.1mM),KH2PO4(3) Na2HPO4(6.78),M63,MS57,MgSO4(1mM),glucose(2),E. coli K-12 MG1655 pCA24N-fucO,chloramphenicol(25),ethylene glycol(30mM),JCL260,glucose(3),isobutanol(8g/L),Δpgi glucose M9 pre-evolved,putrescine,ΔsdhCDAB glucose M9 pre-evolved,Δgnd glucose M9 pre-evolved,gentamycin(0.03),(NH4)2SO4(2),(NH4)2SO4(2) MgSO4(0.12),C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins,C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins,KH2PO4(13.6),none,propanediol,ΔptsHI-crr glucose M9 pre-evolved,Δpgi,LB,m-tartrate,doxycycline,ceftazidime,pBGT,P. aerophilum tpiA,glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1),ΔsdaA ΔsdaB ΔtdcG ΔglyA,D-2-deoxyribose,bla-TEM-1,ΔtpiA,trimethoprim
nagA,31.694192,29.964408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
clsA,0.000000,0.000000,48.234483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph-pyrE attenuator terminator,0.000000,0.000000,7.426796,1.730867,11.118027,7.406031,4.631097,4.538106,1.712513,1.730867,1.712513,1.281816,10.627396,12.653294,1.781367,3.42121,5.797233,1.755294,5.646967,12.102975,5.557801,5.296502,4.287962,12.828866,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph,0.000000,0.000000,14.581493,1.736214,21.778650,14.496343,0.000000,7.013165,1.688618,1.736214,1.688618,0.000000,6.947426,0.000000,1.773939,0.00000,8.898858,2.432301,0.000000,8.361948,0.000000,0.000000,0.000000,25.129936,7.496595,7.496595,2.867032,7.496595,7.496595,7.496595,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rpoC,5.802921,0.000000,7.302994,1.690775,0.000000,0.000000,0.000000,0.000000,1.704287,1.690775,1.704287,1.303272,0.000000,0.000000,1.813101,0.00000,0.000000,0.000000,5.236176,0.000000,0.000000,0.000000,0.000000,19.787532,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.460357,4.432252,9.01394,4.944095,7.792835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [7]:
# remove columns and rows of just 0s

signif_odds_assoc_mat = signif_odds_assoc_mat.loc[:, (signif_odds_assoc_mat != 0).any(axis=0)]  # remove all columns with only 0s
signif_odds_assoc_mat = signif_odds_assoc_mat.loc[(signif_odds_assoc_mat != 0).any(axis=1)]
signif_odds_assoc_mat

,glutaric acid,ΔtpiA glucose M9 pre-evolved,42 celsius,CaCl2(0.1),FeSO4(20uM) sodium citrate(10mM),GMOS,H. sapien pgi,H. sapien tpiA,KH2PO4(3) Na2HPO4(6.8),M9,MgSO4(0.24),NH4Cl(1),U-13C glucose(2),V. cholerae tpiA,WT,adipic acid,chloramphenicol,glucose(4),glucose(4) or acetate(4),glucose(4) or glycerol(2.5),glucose(4) or xylose(4),hexamethylenediamine,n-butanol,xylose(4),(NH4)2SO4(1),(NH4)2SO4(1) MgSO4,30 celsius,K2HPO4(5.3) KH2PO4(2),glucose(1),thiaminne hydrochloride(0.3µM) Ga(NO3)3(100mg/L),2 3-butanediol,B. melitensis pgi,V. cholerae pgi,hexanoic acid,octanoic acid,C13H10N2O(0.168184),isobutyric acid,coumaric acid,D-arabinose,acetate(4),PQ(0.8mM),glucose-evolved BOP1000,D-lyxose,NaCl(0.5g/L) trace elements,P. aerophilum pgi,monomethyl succinate,glycerol(2.5),CaCl2(0.1mM),KH2PO4(3) Na2HPO4(6.78),M63,MS57,MgSO4(1mM),glucose(2),E. coli K-12 MG1655 pCA24N-fucO,chloramphenicol(25),ethylene glycol(30mM),JCL260,glucose(3),isobutanol(8g/L),Δpgi glucose M9 pre-evolved,putrescine,ΔsdhCDAB glucose M9 pre-evolved,Δgnd glucose M9 pre-evolved,gentamycin(0.03),(NH4)2SO4(2),(NH4)2SO4(2) MgSO4(0.12),C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins,C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins,KH2PO4(13.6),none,propanediol,ΔptsHI-crr glucose M9 pre-evolved,Δpgi,LB,m-tartrate,doxycycline,ceftazidime,pBGT,P. aerophilum tpiA,glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1),ΔsdaA ΔsdaB ΔtdcG ΔglyA,D-2-deoxyribose,bla-TEM-1,ΔtpiA,trimethoprim
nagA,31.694192,29.964408,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
clsA,0.000000,0.000000,48.234483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph-pyrE attenuator terminator,0.000000,0.000000,7.426796,1.730867,11.118027,7.406031,4.631097,4.538106,1.712513,1.730867,1.712513,1.281816,10.627396,12.653294,1.781367,3.42121,5.797233,1.755294,5.646967,12.102975,5.557801,5.296502,4.287962,12.828866,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph,0.000000,0.000000,14.581493,1.736214,21.778650,14.496343,0.000000,7.013165,1.688618,1.736214,1.688618,0.000000,6.947426,0.000000,1.773939,0.00000,8.898858,2.432301,0.000000,8.361948,0.000000,0.000000,0.000000,25.129936,7.496595,7.496595,2.867032,7.496595,7.496595,7.496595,0.000000,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rpoC,5.802921,0.000000,7.302994,1.690775,0.000000,0.000000,0.000000,0.000000,1.704287,1.690775,1.704287,1.303272,0.000000,0.000000,1.813101,0.00000,0.000000,0.000000,5.236176,0.000000,0.000000,0.000000,0.000000,19.787532,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.460357,4.432252,9.01394,4.944095,7.792835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [8]:
import numpy as np
signif_logodds_assoc_mat = np.log(signif_odds_assoc_mat)
signif_logodds_assoc_mat = signif_logodds_assoc_mat.replace(-np.inf, 0)
signif_logodds_assoc_mat

/home/pphaneuf/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


,glutaric acid,ΔtpiA glucose M9 pre-evolved,42 celsius,CaCl2(0.1),FeSO4(20uM) sodium citrate(10mM),GMOS,H. sapien pgi,H. sapien tpiA,KH2PO4(3) Na2HPO4(6.8),M9,MgSO4(0.24),NH4Cl(1),U-13C glucose(2),V. cholerae tpiA,WT,adipic acid,chloramphenicol,glucose(4),glucose(4) or acetate(4),glucose(4) or glycerol(2.5),glucose(4) or xylose(4),hexamethylenediamine,n-butanol,xylose(4),(NH4)2SO4(1),(NH4)2SO4(1) MgSO4,30 celsius,K2HPO4(5.3) KH2PO4(2),glucose(1),thiaminne hydrochloride(0.3µM) Ga(NO3)3(100mg/L),2 3-butanediol,B. melitensis pgi,V. cholerae pgi,hexanoic acid,octanoic acid,C13H10N2O(0.168184),isobutyric acid,coumaric acid,D-arabinose,acetate(4),PQ(0.8mM),glucose-evolved BOP1000,D-lyxose,NaCl(0.5g/L) trace elements,P. aerophilum pgi,monomethyl succinate,glycerol(2.5),CaCl2(0.1mM),KH2PO4(3) Na2HPO4(6.78),M63,MS57,MgSO4(1mM),glucose(2),E. coli K-12 MG1655 pCA24N-fucO,chloramphenicol(25),ethylene glycol(30mM),JCL260,glucose(3),isobutanol(8g/L),Δpgi glucose M9 pre-evolved,putrescine,ΔsdhCDAB glucose M9 pre-evolved,Δgnd glucose M9 pre-evolved,gentamycin(0.03),(NH4)2SO4(2),(NH4)2SO4(2) MgSO4(0.12),C6H5FeO7(0.00049) casamino(5) WT P. aeruginosa toxins,C6H5FeO7(0.00049) casamino(5) pvdJ KO P. aeruginosa toxins,KH2PO4(13.6),none,propanediol,ΔptsHI-crr glucose M9 pre-evolved,Δpgi,LB,m-tartrate,doxycycline,ceftazidime,pBGT,P. aerophilum tpiA,glycine(2mM) L-Serine(varying) Wolfe's vitamin solution trace elements(X1),ΔsdaA ΔsdaB ΔtdcG ΔglyA,D-2-deoxyribose,bla-TEM-1,ΔtpiA,trimethoprim
nagA,3.456133,3.40001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
clsA,0.000000,0.00000,3.876074,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph-pyrE attenuator terminator,0.000000,0.00000,2.005095,0.548622,2.408568,2.002295,1.532794,1.512510,0.537962,0.548622,0.537962,0.248278,2.363435,2.537918,0.577381,1.229994,1.757381,0.562637,1.731119,2.493451,1.715203,1.667047,1.455812,2.551698,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rph,0.000000,0.00000,2.679753,0.551707,3.080930,2.673896,0.000000,1.947789,0.523910,0.551707,0.523910,0.000000,1.938371,0.000000,0.573202,0.000000,2.185923,0.888838,0.000000,2.123691,0.000000,0.000000,0.000000,3.224060,2.014449,2.014449,1.053277,2.014449,2.014449,2.014449,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
rpoC,1.758361,0.00000,1.988284,0.525187,0.000000,0.000000,0.000000,0.000000,0.533147,0.525187,0.533147,0.264878,0.000000,0.000000,0.595039,0.000000,0.000000,0.000000,1.655592,0.000000,0.000000,0.000000,0.000000,2.985052,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.495229,1.488908,2.198772,1.598194,2.053205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [9]:
signif_logodds_assoc_mat.to_pickle("../data/assocs/geno_signif_logodds_assoc_mat.pkl")